In [4]:
#Importing essential libraries
from sklearn.datasets import fetch_openml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os 
import cv2
import random 
import math

from datetime import datetime

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow import keras
import pickle

In [5]:
pickle_in = open("X_train_cic.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y_train_cic.pickle","rb")
y_train = pickle.load(pickle_in)

pickle_in = open("X_test_cic.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("y_test_cic.pickle","rb")
y_test = pickle.load(pickle_in)

In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1398983, 70)
(1398983,)
(749456, 70)
(749456,)


In [30]:
array_y_train=np.array(y_train)


In [43]:
np.unique(array_y_train)

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=uint32)

In [7]:
#Spliting the training data into validation set (20%)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size = 0.25, random_state = 42)
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

(1049237, 70) (349746, 70)
(1049237,) (349746,)


In [8]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

model = keras. models.Sequential()
model.add(keras.layers.Dense(21, activation ="sigmoid"))
#model.add(keras.layers.Dense(300, activation ="sigmoid"))
#model.add(keras.layers.Dense(300, activation ="sigmoid"))
#model.add(keras.layers.Dense(300, activation ="relu"))
#model.add(keras.layers.Dense(300, activation ="sigmoid"))
model.add(keras.layers.Dense(8, activation = "softmax"))


#Defining callbacks with ALR, Early stopping , Tensorboard
early_stopping_monitor = EarlyStopping(patience=10)
#callbacks_list = [early_stopping_monitor,tensorboard,lrate]
#callbacks_list = [lrate]

sgd = SGD(learning_rate=0.001)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

X_train = X_train.astype('float32')
y_train = y_train.astype('uint')
X_val = X_val.astype('float32')
y_val = y_val.astype('uint')

#model.fit(X_train, y_train, batch_size=128, epochs=150, validation_data =(X_val,y_val) ,  callbacks =callbacks_list)
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data =(X_val,y_val))

Epoch 1/50
8198/8198 [==============================] - 11s 1ms/step - loss: 0.3692 - accuracy: 0.8687 - val_loss: 0.3285 - val_accuracy: 0.8739
Epoch 2/50
8198/8198 [==============================] - 11s 1ms/step - loss: 0.3303 - accuracy: 0.8816 - val_loss: 0.3421 - val_accuracy: 0.8753
Epoch 3/50
8198/8198 [==============================] - 11s 1ms/step - loss: 0.3266 - accuracy: 0.8828 - val_loss: 0.2804 - val_accuracy: 0.8858
Epoch 4/50
8198/8198 [==============================] - 10s 1ms/step - loss: 0.3118 - accuracy: 0.8862 - val_loss: 0.3117 - val_accuracy: 0.8838
Epoch 5/50
8198/8198 [==============================] - 11s 1ms/step - loss: 0.3563 - accuracy: 0.8784 - val_loss: 0.3702 - val_accuracy: 0.8748
Epoch 6/50
8198/8198 [==============================] - 11s 1ms/step - loss: 0.3458 - accuracy: 0.8789 - val_loss: 0.3395 - val_accuracy: 0.8795
Epoch 7/50
8198/8198 [==============================] - 10s 1ms/step - loss: 0.3373 - accuracy: 0.8760 - val_loss: 0.3118 - val_ac

In [9]:
X_test = X_test.astype('float32')
y_test = y_test.astype('uint')


loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.3f' % acc)
err=1-acc
print('Misclassification Error: %.3f' % err)


Accuracy: 0.890
Misclassification Error: 0.110


In [10]:
from sklearn.metrics import classification_report, confusion_matrix

y_proba = model.predict(X_test) 
y_pred = model.predict_classes(X_test)
print('Confusion Matrix')
print(confusion_matrix(y_pred, y_test))
print('Classification Report')
print(classification_report(y_pred, y_test))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Confusion Matrix
[[605876  25684  14115  11847   2755    671    560      9]
 [  7535  27890     11   6016      0      0      0      0]
 [   532      0  12803      0      0      0      0      0]
 [  8246   4257      2  20647      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]]
Classification Report


c:\users\rathi\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      0.92      0.94    661517
           1       0.48      0.67      0.56     41452
           2       0.48      0.96      0.64     13335
           3       0.54      0.62      0.58     33152
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0

    accuracy                           0.89    749456
   macro avg       0.31      0.40      0.34    749456
weighted avg       0.92      0.89      0.90    749456

